# Big project activity

## Optimal chargin station location

### 1.Introduction

Consider a long linear cycle path  as Vento, or the Danube cycle path. The cycle path usually runs along the banks of a river with scarse tourist interest. However, from the main course of the cycle path it is possible to reach places of tourist interest by making small detours.  

The rapid growth of e-bike ridership is proposing the problem of deploying a suitable charging infrastructure. The charging stations should be placed in strategic positions so as to guarantee a coverage of the whole cycle path. However, since the charging operations require a non negligible time, the charging station should be positioned in places where alternative activities could be carried out, as restaurants, museums, swimming pool, or other amenities. Moreover, the presence of a charging station could also induce e-cyclists to discover new places and generate positive externalities.



### 2.Decision problem
We can represent the cycle path as a graph where the set of nodes $H = \{1,\ldots,n\}$ corresponds to the tourist sites that may host a charging station.
In addition we are given the distances between touristic site ($d_{ij},$ with $ i,j =1,\ldots,n$). Let $c_i$ be the cost of installing a charging station in site $i, i=1\ldots,n$.


The problem is, given a budget $b$, determine the subset of sites $S\subseteq H$ where to install the charging stations so that the total cost is not higher than $b$ and the maximum distance between consecutive charging stations is minimized.
Consider that the cyclist has to visit all the touristic destination in a consecutive way.



### 3.Problem characteristics
There are 2 csv files that contain the information of the cycle way, they are essential to build the equivalent graph:


*    in the "nodes.csv" file there are all the destinations that the cyclist can reach, with their spatial coordinates and the value of installation costs related to that destination. Consider that the "tourist-dest-id" is not the graph node number but it is an unique id to identify the destination.
*   in the "OD.csv" you can find all the arcs between two different nodes, keep attention that the condition of visiting consecutive touristic destination must be respected.

The set of nodes $N$ is defined by $\{0,1,\ldots,n, n+1\}$.  The Arcs $A$ correspond to the portion of cycle path between two consecutive charging stations. We assume that potentially e-riders will visit all sites along the way making the suitable deviations and going back to the main path at the initial point of the detour.
The cost associated with each  arc $(i,j)$ is given by $c_j$, thus the cost of installing a charging station in $j$. These costs are defined for all arcs in $A$, while they are set to 0 for all the arcs that arrive in the last node.
The path starts in node $s = 0$ and ends in node $t = n+1$, these two nodes are connected to the nearest touristic site with an arc of null length.

### 4.Example of linear path with deviation
![picture](https://drive.google.com/uc?export=view&id=1w16bHtbu0FGGL-UntxeqxD7244D3eHbJ)

### 5.Requirements
The requirements of the problem are:


*   the maximum running time of the algorithm must be 5 minutes, so set the proper timer
*   create the equivalent graph and display it on a xy-plot
*   find the solution for the basic scenario, with a mip model, displaying the solution with a xy-plot, the budget constrain is $b = 10000\ € $.
*   Find the optimal solution for 5 different values of budget in the range $[10000, 100000]$. Select the values of the budget so as to have different charger locations.

  You have to motivate you choice and the solution you get. They can also be not common solution if they are well motivated. To support your decision and explanations you can print plots or tables. You can also compare different scenarios.


   
If you have some doubts related to the parametric analysis prof. Cubillos uploaded a notebook with the solution on WeBeeP and you can have a look there.

### Insert student name and student ID

student1:

ID1:

student2:

ID2:

student3

ID3:



In [16]:
#install libraries

#!pip install mip
#!pip install --upgrade cffi==1.15.0

In [17]:
#import libraries
import mip
import pandas as pd
import importlib
import cffi
importlib.reload(cffi)
import numpy as np
import math
import networkx as nx
import matplotlib as pl

In [18]:
#import the csv file
nodes_df = pd.read_csv("./nodes.csv", encoding='cp1252')
arcs_df = pd.read_csv("./OD.csv")

nodes_df.head()
arcs_df.head()

,origin_id,destination_id,distance [m]
0,0,0,NaN
1,0,17,43798.08450
2,0,18,47252.99168
3,0,20,57171.88493
4,0,21,62260.57345


In [19]:
#set the timer

#TO DO

In [38]:
#build the equivalent graph

#define the set of nodes
N = set()
for index, node in nodes_df.iterrows():
    N.add(node.tourist_dest_id)

n = len(N)
s = 0
t = 90

#define the set of edges
def d(i, j):
    res = arcs_df.loc[(arcs_df['origin_id'] == i) & (arcs_df['destination_id'] == j), 'distance [m]'].values[0]
    return res

def c(i, j):
    if j == t: return 0
    return nodes_df.loc[(nodes_df['tourist_dest_id'] == j), 'Cost_of_installation [€]'].values[0]

A = set()
for idx, arc in arcs_df.iterrows():
    i = int(arc.origin_id)
    j = int(arc.destination_id)
    if i != j: A.add((i, j))
    
A_dict = {(i, j): {'weight': c(i, j)} for i, j in A}


#plot the graph
G = nx.DiGraph()
G.add_nodes_from(N)
G.add_edges_from(A_dict)
pos = {id: (pos_x, pos_y) for id, pos_x, pos_y in nodes_df[['tourist_dest_id', 'x (longitude)', 'y (latitude)']].values}
#nx.draw(G, pos, labels={node: node for node in G.nodes()})

# TSP
nx.approximation.traveling_salesman_problem(G, nodes=[0, 52] ,weight='weight', cycle=False)

tsp = nx.approximation.traveling_salesman_problem
G1 = nx.cycle_graph(9)
G1[4][5]["weight"] = 5  # all other weights are 1
tsp(G1, nodes=[3, 6], cycle=False)


NetworkXError: G must have at least two nodes

In [21]:
import mip
# Create model
m = mip.Model()

# define the variables
x = {(i, j): m.add_var(var_type=mip.BINARY) for (i, j) in A}            # decisions: arc (i, j) belongs to the path
D = m.add_var()                                                         # dummy variable for min-max linearization

#  define the contraints

for i in N:
  if i != s and i != t:
    m.add_constr( mip.xsum(x[(i, j)] for i, j in A) == 2 )           # two arcs for each node in the path

from itertools import chain, combinations
def powerset(iterable):
  s = list(iterable)
  return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))
for S in powerset(N):
  m.add_constr( mip.xsum(x[(i, j)] for (i, j) in A if i in S and j not in S) >= 1)  # connectivity of sets constraint

'''b = {}
for idx, node in nodes_df.iterrows():
  id = node[0]
  if id == s: b[id] = -1
  elif id == t: b[id] = 1
  else: b[id] = 0

for i in N:
  m.add_constr(
      mip.xsum(x[(j, i)] for j in N if (j, i) in A) -
      mip.xsum(x[(i, j)] for j in N if (i, j) in A) == b[i] )           # shortest path connectivity constraint
  '''
budget = 10000
m.add_constr( mip.xsum(x[(i, j)] * c(i, j) for i, j in A) <= budget )   # budget constraint

for (i, j) in A:
  m.add_constr( D >= x[(i, j)] * d(i, j) )                              # dummy variable constraint

# optimize objective function
m.objective = mip.minimize( D )

m.optimize()
print(D.x)

ValueError: too many values to unpack (expected 2)

In [ ]:
print(m.objective_value)

16011.18979


In [ ]:
#plot the graph

In [ ]:
# parametric analysis



#TO DO


# SPT su grafo da 0 al nodo tc è minimizzato il path
# SPT come fatto in precedenza su grafo ordinato topologicamente come al punto precedente
